# Getting Started with MLFlow for LLM Evaluation
MLflow is a powerful open-source platform for managing the machine learning lifecycle. While it's traditionally used for tracking model experiments, logging parameters, and managing deployments, MLflow has recently introduced support for evaluating Large Language Models (LLMs).

In this tutorial, we explore how to use MLflow to evaluate the performance of an LLM—in our case, Google's Gemini model—on a set of fact-based prompts. We'll generate responses to fact-based prompts using Gemini and assess their quality using a variety of metrics supported directly by MLflow.

## 1. Setting up the dependencies
For this tutorial, we’ll be using both the OpenAI and Gemini APIs. MLflow’s built-in generative AI evaluation metrics currently rely on OpenAI models (e.g., GPT-4) to act as judges for metrics like answer similarity or faithfulness, so an OpenAI API key is required. You can obtain:

* Your OpenAI API key from https://platform.openai.com/settings/organization/api-keys

* Your Google Gemini API key from https://ai.google.dev/gemini-api/docs

### 1.1 Installing the libraries

In [1]:
!pip install mlflow openai pandas google-genai

### 1.2 Setting the OpenAI and Google API Keys as environment variable

In [2]:
import os
from getpass import getpass

os.environ["OPENAI_API_KEY"] = getpass('Enter OpenAI API Key:')
os.environ["GOOGLE_API_KEY"] = getpass('Enter Google API Key:')

Enter OpenAI API Key:··········
Enter Google API Key:··········


## 2. Preparing Evaluation Data and Fetching Outputs from Gemini

### 2.1 Importing the libraries

In [3]:
import mlflow
import openai
import os
import pandas as pd
from google import genai

### 2.2 Creating the evaluation data
In this step, we define a small evaluation dataset containing factual prompts along with their correct ground truth answers. These prompts span topics such as science, health, web development, and programming. This structured format allows us to objectively compare the Gemini-generated responses against known correct answers using various evaluation metrics in MLflow.

In [4]:
eval_data = pd.DataFrame(
    {
        "inputs": [
            "Who developed the theory of general relativity?",
            "What are the primary functions of the liver in the human body?",
            "Explain what HTTP status code 404 means.",
            "What is the boiling point of water at sea level in Celsius?",
            "Name the largest planet in our solar system.",
            "What programming language is primarily used for developing iOS apps?",
        ],
        "ground_truth": [
            "Albert Einstein developed the theory of general relativity.",
            "The liver helps in detoxification, protein synthesis, and production of biochemicals necessary for digestion.",
            "HTTP 404 means 'Not Found' — the server can't find the requested resource.",
            "The boiling point of water at sea level is 100 degrees Celsius.",
            "Jupiter is the largest planet in our solar system.",
            "Swift is the primary programming language used for iOS app development."
        ]
    }
)

eval_data

,inputs,ground_truth
0,Who developed the theory of general relativity?,Albert Einstein developed the theory of genera...
1,What are the primary functions of the liver in...,"The liver helps in detoxification, protein syn..."
2,Explain what HTTP status code 404 means.,HTTP 404 means 'Not Found' — the server can't ...
3,What is the boiling point of water at sea leve...,The boiling point of water at sea level is 100...
4,Name the largest planet in our solar system.,Jupiter is the largest planet in our solar sys...
5,What programming language is primarily used fo...,Swift is the primary programming language used...


### 2.3 Getting Gemini Responses
This code block defines a helper function gemini_completion() that sends a prompt to the Gemini 1.5 Flash model using the Google Generative AI SDK and returns the generated response as plain text. We then apply this function to each prompt in our evaluation dataset to generate the model's predictions, storing them in a new "predictions" column. These predictions will later be evaluated against the ground truth answers

In [5]:
client = genai.Client()
def gemini_completion(prompt: str) -> str:
    response = client.models.generate_content(
        model="gemini-1.5-flash",
        contents=prompt
    )
    return response.text.strip()

eval_data["predictions"] = eval_data["inputs"].apply(gemini_completion)
eval_data

,inputs,ground_truth,predictions
0,Who developed the theory of general relativity?,Albert Einstein developed the theory of genera...,Albert Einstein developed the theory of genera...
1,What are the primary functions of the liver in...,"The liver helps in detoxification, protein syn...",The liver performs a vast array of crucial fun...
2,Explain what HTTP status code 404 means.,HTTP 404 means 'Not Found' — the server can't ...,HTTP status code 404 means that the server cou...
3,What is the boiling point of water at sea leve...,The boiling point of water at sea level is 100...,100 degrees Celsius
4,Name the largest planet in our solar system.,Jupiter is the largest planet in our solar sys...,Jupiter
5,What programming language is primarily used fo...,Swift is the primary programming language used...,Swift is the primary programming language used...


## 3. Evaluating Gemini Outputs with MLflow
In this step, we initiate an MLflow run to evaluate the responses generated by the Gemini model against a set of factual ground-truth answers. We use the mlflow.evaluate() method with four lightweight metrics: **answer_similarity** (measuring semantic similarity between the model’s output and the ground truth), **exact_match** (checking for word-for-word matches), **latency** (tracking response generation time), and **token_count** (logging the number of output tokens).

It's important to note that the **answer_similarity** metric internally uses OpenAI’s GPT model to judge the semantic closeness between answers, which is why access to the OpenAI API is required. This setup provides an efficient way to assess LLM outputs without relying on custom evaluation logic. The final evaluation results are printed and also saved to a CSV file for later inspection or visualization.

In [6]:
mlflow.set_tracking_uri("mlruns")
mlflow.set_experiment("Gemini Simple Metrics Eval")

with mlflow.start_run():
    results = mlflow.evaluate(
        model_type="question-answering",
        data=eval_data,
        predictions="predictions",
        targets="ground_truth",
        extra_metrics=[
          mlflow.metrics.genai.answer_similarity(),
          mlflow.metrics.exact_match(),
          mlflow.metrics.latency(),
          mlflow.metrics.token_count()
      ]
    )
    print("Aggregated Metrics:")
    print(results.metrics)

    # Save detailed table
    results.tables["eval_results_table"].to_csv("gemini_eval_results.csv", index=False)

2025/06/26 18:18:38 WARNING mlflow.models.evaluation.evaluators.default: Setting the latency to 0 for all entries because the model is not provided.
2025/06/26 18:18:38 INFO mlflow.models.evaluation.default_evaluator: Testing metrics on first row...
2025/06/26 18:18:39 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/06/26 18:18:39 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/06/26 18:18:39 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/06/26 18:18:39 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/06/26 18:18:39 WA

  0%|          | 0/1 [00:00<?, ?it/s]

2025/06/26 18:18:43 WARNING mlflow.metrics.metric_definitions: Failed to load 'toxicity' metric (error: ModuleNotFoundError("No module named 'evaluate'")), skipping metric logging.
2025/06/26 18:18:43 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'toxicity' at index 1 in the `extra_metrics` parameter because it returned None.
2025/06/26 18:18:43 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for flesch kincaid metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/06/26 18:18:43 WARNING mlflow.models.evaluation.utils.metric: Did not log metric 'flesch_kincaid_grade_level' at index 2 in the `extra_metrics` parameter because it returned None.
2025/06/26 18:18:43 WARNING mlflow.metrics.metric_definitions: Failed to import textstat for automated readability index metric, skipping metric logging. Please install textstat using 'pip install textstat'.
2025/06/26 18:18:43 WARNING mlflow.models.evaluation.utils.metri

  0%|          | 0/6 [00:00<?, ?it/s]

Aggregated Metrics:
{'latency/mean': np.float64(0.0), 'latency/variance': np.float64(0.0), 'latency/p90': np.float64(0.0), 'exact_match/v1': 0.16666666666666666, 'answer_similarity/v1/mean': np.float64(4.333333333333333), 'answer_similarity/v1/variance': np.float64(2.2222222222222223), 'answer_similarity/v1/p90': np.float64(5.0)}


To view the detailed results of our evaluation, we load the saved CSV file into a DataFrame and adjust the display settings to ensure full visibility of each response. This allows us to inspect individual prompts, Gemini-generated predictions, ground truth answers, and the associated metric scores without truncation, which is especially helpful in notebook environments like Colab or Jupyter.

In [8]:
results = pd.read_csv('gemini_eval_results.csv')
pd.set_option('display.max_colwidth', None)
results

,inputs,ground_truth,predictions,latency,token_count,answer_similarity/v1/score,answer_similarity/v1/justification
0,Who developed the theory of general relativity?,Albert Einstein developed the theory of general relativity.,Albert Einstein developed the theory of general relativity.,0,10,5,"The provided output exactly matches the target, indicating a perfect semantic similarity. The output and target both state that ""Albert Einstein developed the theory of general relativity,"" demonstrating complete alignment in all significant aspects."
1,What are the primary functions of the liver in the human body?,"The liver helps in detoxification, protein synthesis, and production of biochemicals necessary for digestion.","The liver performs a vast array of crucial functions in the human body. These can be broadly categorized as follows:\n\n**1. Metabolism:**\n\n* **Carbohydrate metabolism:** The liver regulates blood glucose levels by storing glucose as glycogen (glycogenesis), breaking down glycogen into glucose (glycogenolysis), and producing glucose from non-carbohydrate sources (gluconeogenesis).\n* **Lipid metabolism:** The liver synthesizes cholesterol and lipoproteins, processes fatty acids, and helps in the breakdown of fats.\n* **Protein metabolism:** The liver synthesizes many essential proteins, including albumin (important for blood osmotic pressure), clotting factors, and various enzymes. It also deaminates amino acids (removes nitrogen) and converts ammonia (toxic) to urea (less toxic for excretion by the kidneys).\n\n**2. Detoxification:**\n\n* **Drug metabolism:** The liver breaks down and metabolizes many medications and other substances, making them less harmful or easier to excrete.\n* **Waste product removal:** The liver processes and eliminates various metabolic waste products, toxins, and drugs from the bloodstream.\n* **Hormone metabolism:** The liver metabolizes and inactivates many hormones.\n\n**3. Excretion:**\n\n* **Bile production:** The liver produces bile, a fluid crucial for fat digestion and absorption in the intestines. Bile contains bile salts, bilirubin (a byproduct of hemoglobin breakdown), and cholesterol. Bilirubin is then excreted in the feces.\n\n**4. Storage:**\n\n* **Glycogen storage:** As mentioned above, the liver stores glucose as glycogen.\n* **Vitamin and mineral storage:** The liver stores vitamins A, D, E, K, and B12, as well as iron.\n\n**5. Synthesis:**\n\n* **Protein synthesis:** The liver synthesizes many essential proteins vital for various bodily functions.\n* **Bile synthesis:** As mentioned above.\n* **Plasma protein synthesis:** Including albumin and clotting factors.\n\n\nThese are the primary functions; the liver's roles are interconnected and essential for maintaining overall health and homeostasis. Damage to the liver can have severe and wide-ranging consequences throughout the body.",0,475,5,"The model's output closely aligns with the provided targets in all significant aspects. It covers the liver's role in detoxification, protein synthesis, and production of biochemicals necessary for digestion, which are the key points in the targets. Additionally, it provides a comprehensive and detailed explanation of these functions, demonstrating a high degree of semantic similarity."
2,Explain what HTTP status code 404 means.,HTTP 404 means 'Not Found' — the server can't find the requested resource.,"HTTP status code 404 means that the server could not find the requested resource. This is usually what people see when they get a ""Page Not Found"" error on a website. The client (like a web browser) sent a request for a specific page or file, but the server couldn't locate it at the specified URL.",0,68,5,"The provided output closely aligns with the target in all significant aspects. It accurately explains that HTTP 404 means 'Not Found' and that it is related to the server not being able to find the requested resource. The output also provides additional context and details, such 